In [1]:
import csv
import json
import pprint
import urllib.request
import gzip
import io
import urllib
import pandas as pd
import requests
import numpy as np

In [102]:


path1 = 'https://api.brand.cash/uploads/mevlowadecsapjin/receipt_1000_000.csv.gz'
response = requests.get(path1)
f = gzip.open(io.BytesIO(response.content), mode="rb") 
csvreader = csv.reader(codecs.iterdecode(f, 'utf-8'), delimiter=',', quotechar='"')
def generate_row(row):
    #for row in csvreader:
    result = pd.DataFrame()
    data = json.loads(row[3])
    user = data.get("user")
    items = data.get("items", [])
    for item in items:
        tmp = dict()
        tmp["user"] = user
        tmp["sum"] = item.get("sum")
        tmp["name"] = item.get("name")
        tmp["price"] = item.get("price")
        result = result.append(pd.DataFrame.from_dict(tmp, orient='index').T, ignore_index=True)
    return result


In [105]:
import time
start_time=time.time()
full=pd.DataFrame()
from joblib import Parallel, delayed
full=Parallel(n_jobs=5)(delayed(generate_row)(row) for row in csvreader)
print("--- %s seconds ---" % (time.time() - start_time)) 

--- 7378.906464338303 seconds ---


In [141]:
result = pd.concat(full).reset_index()

In [143]:
result = result.drop('index', axis=1)

In [145]:
result.to_csv('names_product.csv')

In [2]:
result = pd.read_csv('names_product.csv').drop('Unnamed: 0', axis=1)

In [3]:
result = result[result['name'].notna()]

In [7]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8012834 entries, 0 to 8013805
Data columns (total 4 columns):
 #   Column  Dtype  
---  ------  -----  
 0   user    object 
 1   sum     float64
 2   name    object 
 3   price   float64
dtypes: float64(2), object(2)
memory usage: 305.7+ MB


In [8]:
result['name'].nunique()

1394015

In [156]:
result['len'] = result['name'].apply(len)

In [162]:
print(result['len'].max(), result['len'].mean(), result['len'].quantile(0.95))

312 34.39758289073281 50.0


## Build word 2 vec model for names

In [25]:
result['name_list']=result['name'].apply(preprocess)

In [26]:
result.head()

,user,sum,name,price,name_list
0,NaN,3856.0,Слива 1кг,9990.0,"[слива, 1кг]"
1,NaN,3990.0,Майонез Махеев Оливковый д/п 1/400мл,3990.0,"[майонез, махеев, оливковый, д, п, 1, 400мл]"
2,NaN,15251.0,Палочки крабовые Эконом зам вес 1кг,18690.0,"[палочки, крабовые, эконом, зам, вес, 1кг]"
3,NaN,4290.0,Сметана Молком 15% м.д.ж. пл/ст 1/20,4290.0,"[сметана, молком, 15%, м.д.ж., пл, ст, 1, 20]"
4,NaN,3649.0,Арахис соленый 1кг,23390.0,"[арахис, соленый, 1кг]"


In [9]:
import time
start_time=time.time()
from joblib import Parallel, delayed
full=Parallel(n_jobs=5)(delayed(preprocess)(name) for name in (result['name']))
print("--- %s seconds ---" % (time.time() - start_time)) 

--- 192.3453140258789 seconds ---


In [10]:
full[:20]

[['слива', '1кг'],
 ['майонез', 'махеев', 'оливковый', 'д', 'п', '1', '400мл'],
 ['палочки', 'крабовые', 'эконом', 'зам', 'вес', '1кг'],
 ['сметана', 'молком', '15%', 'м.д.ж.', 'пл', 'ст', '1', '20'],
 ['арахис', 'соленый', '1кг'],
 ['рулет', 'мексиканский', 'в', 'к', '1кг', 'мяснмаст'],
 ['приправа', 'кладовая', 'вкуса', 'кунжут', '1', '10г'],
 ['нектарины', '1кг'],
 ['паста', 'крем-фиш', 'треска', 'пл', 'ст', '1', '150гр'],
 ['*3147882', 'яйцо', 'kind.sur.м.шок', 'игр.20г'],
 ['*3147882', 'яйцо', 'kind.sur.м.шок', 'игр.20г'],
 ['53715', 'fer.пир.киндер', 'макси', 'кинг', '35г'],
 ['*3427052', 'свитлог.сырок', 'тв.гл.ван.26%50г'],
 ['4069735', 'свитл.сырок', 'з.ирис.тв.кар.гл50г'],
 ['*53713', 'fer.пир.киндер', 'пингви', '30г'],
 ['147805', 'трусы', 'женские', 'cle', 'b650', '44,', 'черный'],
 ['батон', 'летний', 'в', 'с', '0,35кг', 'не', 'упакхл'],
 ['chipicao', 'круассан', 'крем', 'какао', '60г', 'п'],
 ['*3958355', 'nesc.кофе', 'gold', 'раст.доб.мол220г'],
 ['*3970786', 'soff.бум.p

In [11]:
import warnings
warnings.filterwarnings("ignore")
from gensim.corpora import WikiCorpus
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

In [12]:
import time
start_time=time.time()
model_cbow = Word2Vec(full, size=500, window=30, min_count=1,workers=5, negative=3)
print("--- %s seconds ---" % (time.time() - start_time)) 

--- 844.3444752693176 seconds ---


In [21]:
import pickle
with open('model_names_match.pkl', 'wb') as fid:
    pickle.dump(model_cbow, fid)

In [22]:
model_cbow.most_similar(['майонез', 'махеев', 'оливковый', 'д', 'п', '1', '400мл'])

[('провансаль', 0.607303261756897),
 ('м-з', 0.6053080558776855),
 ('май-з', 0.5910055041313171),
 ('67%', 0.5837064981460571),
 ('махеевъ', 0.5698209404945374),
 ('майонезный', 0.567755937576294),
 ('5:3408630', 0.5646836161613464),
 ('1:3419671', 0.5612503886222839),
 ('13:3419671', 0.5592818260192871),
 ('моск.прованс.домашн.55%', 0.5567857623100281)]

In [19]:
model_cbow['слива' ]+model_cbow['1кг']

array([-9.78727877e-01, -2.84746885e-02,  5.53657055e-01,  3.21929550e+00,
        2.06785131e+00,  2.19968462e+00,  2.74402678e-01,  1.71093392e+00,
        1.06422627e+00,  2.19935894e-01, -2.10816908e+00,  2.73985815e+00,
        2.40936899e+00, -2.21423244e+00, -1.00761414e+00, -1.70314384e+00,
        2.49379158e+00,  1.18441868e+00, -7.36273527e-01,  2.18221450e+00,
        1.10770333e+00, -1.23036671e+00, -4.57368076e-01, -7.75210187e-02,
       -3.29710960e-01,  2.31282687e+00, -1.27458930e+00, -8.32162559e-01,
        1.58426657e-01,  1.30920005e+00, -5.24857640e-01, -1.47250795e+00,
       -1.60249925e+00,  1.95003593e+00, -2.02969000e-01, -1.21950805e-01,
        6.42673969e-02, -1.63989806e+00,  1.45455289e+00, -1.52848649e+00,
       -1.27260268e+00,  1.52751207e+00,  1.56966597e-01,  1.38594687e+00,
        4.73498523e-01, -1.51132345e+00, -7.56590903e-01,  2.31226516e+00,
        2.74986291e+00,  3.12376022e-01,  5.44998288e-01, -1.25842321e+00,
       -1.38310170e+00,  

# Preprocess users

In [2]:
result = pd.read_csv('names_product.csv').drop('Unnamed: 0', axis=1)
result = result[result['name'].notna()]

In [3]:
df_price=result[['user','name']].drop_duplicates(keep='first').reset_index().drop('index', axis=1)

In [4]:
def preprocess_users(text):
    
    text = text.lower()
    if 'дикси' in text:
        result = 'дикси'
    elif 'перекресток'in text:
        result = 'перекресток'
    elif 'тандер' in text:
        result = 'тандер'
    elif 'агроторг' in text:
        result = 'агроторг'
    elif 'виктория' in text:
        result = 'виктория'
    elif 'ип роздухов' in text:
        result = 'ип'
    else:
        result = '-'
    
    return result

In [5]:
df_price= df_price[df_price['user'].notna()]
df_price['user'] =df_price['user'].apply(preprocess_users)


In [6]:
df_price=df_price[df_price['user']!='-']

In [7]:
from string import punctuation
def preprocess(text):
    text=text.replace("/", '')
    text=text.replace("(", '')
    text=text.replace(")", "")
    text = text.lower()
    all_reviews = text
    all_words = text.split(' ')
    all_words = [word for word in all_words if word!='']
    return all_words

In [8]:
df_work = df_price[['user', 'name']].drop_duplicates(keep='first').reset_index().drop('index', axis=1)

In [11]:
df_work['name_list'] = df_work['name'].apply(preprocess)

In [12]:
del(df_price)
del(result)

In [17]:
class matching:
    
    def __init__(self, path_wv, path_net):
        
        import pickle
        
        import torch
        
        import torch.nn.functional as F
        
        from torch import nn
        
        self.path_net = path_net
        
        self.path_wv = path_wv
        
        self.model = pickle.load(open(path_wv, "rb"))
        
        class MatchLSTM(nn.Module):
    
            def __init__(self, n_input, n_hidden, n_output):#,  n_linear,  drop_p = 0.7):
                super().__init__()
        
                self.n_input = n_input
            
                self.n_output = n_output
         
                self.n_hidden = n_hidden
            
                self.lin1 = nn.Linear(n_input, n_hidden)
            
                self.relu = nn.ReLU()
            
                self.fc = nn.Linear(n_hidden, n_output)
            
                self.pool = nn.MaxPool1d( stride=4, dilation=3, kernel_size=20, padding=4)
        
        
            def forward (self, input_x):
                                             
                x = self.lin1(input_x)
            
                x = self.relu(x)
            
                x = self.fc(x)
            
                x = self.pool(x)
        
                out_last = x[:, -1].view(-1, 1)             
        
                return out_last
        
        n_input = 4
        
        n_hidden = 100
        
        n_output = 50   

        net = MatchLSTM(n_input, n_hidden, n_output)

        net.load_state_dict(torch.load(path_net))
    
        self.net = net
        
        
    
    def vector_from_list(self, name_list, model = None):
        
        if model is None:
            
            model = self.model
    
        vector = np.sum([self.model.wv.__getitem__(elem) for elem in name_list], axis=0)
        
        return vector
    
    def cosine_metric(self, vector1, vector2):
        
        metric = np.abs(np.dot(vector1,vector2)/(np.sqrt(vector1.dot(vector1))*np.sqrt(vector2.dot(vector2))))
    
        return metric
    
    
    def sort_data(self, df_part, user):
        
        import torch
        
        self.net.eval()
        
        df_part = df_part[df_part['user'] == user]
        
        df_part['m'] = (df_part['ml'] + df_part['mj'] + df_part['mv'] + df_part['mc'])*0.25

        df_part = df_part.sort_values(by=['m', 'ml', 'mj', 'mv', 'mc'], ascending = False)
        
        X_input = torch.Tensor(df_part[['ml', 'mj', 'mv', 'mc']].values).unsqueeze(dim=1)
        
        predict = self.net(X_input).detach().numpy() 
        
        df_part['mark'] = predict
        
        self.net.train()
            
        return df_part[:5].sort_values(by=[ 'mark'], ascending = False)
    
    def get_user(self, df_part, user):
        
        return df_part[df_part['user'] == user]
    
    
    def init_matching(self, idx, data):
        
        from fuzzywuzzy import fuzz
        
        import textdistance
        
        name = data.loc[idx ,'name']
        
        
        
        vector = self.vector_from_list(data.loc[idx , 'name_list'])
        
        user = data.loc[idx , 'user']
        
        df_part = data[data['user'] != user ]
        
        df_part['ml'] = df_part['name'].apply(lambda x: fuzz.token_sort_ratio(x, name)/100).astype('float16')
        
        df_part = df_part[df_part['ml'] >= 0.4]
        
        df_part['mj'] = df_part['name'].apply(lambda x: textdistance.jaro.normalized_similarity(x, name)).astype('float16')
        
        df_part = df_part[df_part['mj'] >= 0.4]
        
        df_part['vector'] = df_part['name_list'].apply(self.vector_from_list)
        
        df_part['mv'] = df_part['vector'].apply(lambda x: self.cosine_metric(x, vector)).astype('float16')
        
        df_part = df_part[df_part['mv'] >= 0.2]
        
        df_part['mc'] = df_part['name'].apply(lambda x: textdistance.jaccard.normalized_similarity(x, name)).astype('float16')
        
        df_part = df_part[df_part['mc'] >= 0.2]
        
        df_match = pd.DataFrame()
        
        from joblib import Parallel, delayed
        
        match=Parallel(n_jobs=5)(delayed(self.sort_data)(df_part, elem) for elem in df_part['user'].unique())
        
        if len(match)!=0:
            
            df_match = pd.concat(match, ignore_index=True)
        
            df_match['name1'] = name
        
            df_match['user1'] = user
        
        else:
            
            df_match = pd.DataFrame()
        
        return df_match
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        

In [18]:
M=matching('model_names_match.pkl', 'match_nn.pth')

In [19]:
from joblib import Parallel, delayed

import time

start_time=time.time()
        
match = Parallel(n_jobs=5)(delayed(M.init_matching)(i, df_work) for i in range(15001, 16000, 1))

print(time.time()-start_time, 'sec')

29550.09718465805 sec


In [23]:
M.init_matching(3001, df_work)

C:\Users\EkStepanova\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\EkStepanova\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\EkStepanova\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

,user,name,name_list,ml,mj,vector,mv,mc,m,mark,name1,user1
0,агроторг,"*4004 Нап.NEO ИМУНЕЛЕ черник1,2% 100г","[*4004, нап.neo, имунеле, черник1,2%, 100г]",0.629883,0.619141,"[-2.586465, 4.621179, -5.5207605, 3.9616575, 2...",0.620605,0.489746,0.589844,0.852016,"ИМУНЕЛЕ НЕО Напиток к/м черника 1,2%",тандер
1,агроторг,"4004 Нап.NEO ИМУНЕЛЕ черник1,2% 100г","[4004, нап.neo, имунеле, черник1,2%, 100г]",0.629883,0.625000,"[-2.4899373, 4.2752333, -5.4033933, 4.391491, ...",0.618652,0.500000,0.593262,0.851962,"ИМУНЕЛЕ НЕО Напиток к/м черника 1,2%",тандер
2,агроторг,4066651 Напиток ТАН к/м газ.обежир.1л,"[4066651, напиток, тан, к, м, газ.обежир.1л]",0.560059,0.598633,"[2.782912, -0.25235605, -2.6836984, 3.379808, ...",0.864746,0.377441,0.600098,0.820521,"ИМУНЕЛЕ НЕО Напиток к/м черника 1,2%",тандер
3,агроторг,"3993750 Напиток ЛИМОНАД б/а с/г 1,5л","[3993750, напиток, лимонад, б, а, с, г, 1,5л]",0.449951,0.595215,"[-3.132886, 10.313924, -3.381991, 6.961627, 4....",0.677246,0.411865,0.533691,0.820440,"ИМУНЕЛЕ НЕО Напиток к/м черника 1,2%",тандер
4,агроторг,4066650 Напиток АЙРАН к/м газ.обезж.1л,"[4066650, напиток, айран, к, м, газ.обезж.1л]",0.520020,0.546875,"[2.718184, -0.27704072, -2.757209, 3.2017353, ...",0.863281,0.321533,0.562988,0.814537,"ИМУНЕЛЕ НЕО Напиток к/м черника 1,2%",тандер
5,перекресток,"3426770 БЗМЖ Напиток ИМУНЕЛЕ 1,5%","[3426770, бзмж, напиток, имунеле, 1,5%]",0.569824,0.546875,"[0.9791123, 2.606613, -4.0617094, 2.1932123, -...",0.842285,0.489258,0.612305,0.829991,"ИМУНЕЛЕ НЕО Напиток к/м черника 1,2%",тандер
6,перекресток,"*3426770 БЗМЖ Напиток ИМУНЕЛЕ 1,5%","[*3426770, бзмж, напиток, имунеле, 1,5%]",0.569824,0.541992,"[0.9211394, 2.7452586, -4.189718, 1.796814, -3...",0.843262,0.479248,0.608398,0.829847,"ИМУНЕЛЕ НЕО Напиток к/м черника 1,2%",тандер
7,перекресток,3929854 БЗМЖ Напиток к/м ИМУНЕЛЕ,"[3929854, бзмж, напиток, к, м, имунеле]",0.629883,0.525391,"[1.4209044, 0.6371767, -4.257227, 3.3396418, -...",0.938965,0.500000,0.648438,0.829578,"ИМУНЕЛЕ НЕО Напиток к/м черника 1,2%",тандер
8,перекресток,6:3929854 БЗМЖ Напиток к/м ИМУНЕЛЕ,"[6:3929854, бзмж, напиток, к, м, имунеле]",0.609863,0.612793,"[1.4132621, 0.46205354, -4.086008, 3.3765593, ...",0.937500,0.479248,0.659668,0.823296,"ИМУНЕЛЕ НЕО Напиток к/м черника 1,2%",тандер
9,перекресток,4066651 Напиток ТАН к/м газ.обежир.1л,"[4066651, напиток, тан, к, м, газ.обежир.1л]",0.560059,0.598633,"[2.782912, -0.25235605, -2.6836984, 3.379808, ...",0.864746,0.377441,0.600098,0.820521,"ИМУНЕЛЕ НЕО Напиток к/м черника 1,2%",тандер


In [20]:
res = pd.concat(match, ignore_index=True)

In [21]:
res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20822 entries, 0 to 20821
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   user       20822 non-null  object 
 1   name       20822 non-null  object 
 2   name_list  20822 non-null  object 
 3   ml         20822 non-null  float16
 4   mj         20822 non-null  float16
 5   vector     20822 non-null  object 
 6   mv         20822 non-null  float16
 7   mc         20822 non-null  float16
 8   m          20822 non-null  float16
 9   mark       20822 non-null  float32
 10  name1      20822 non-null  object 
 11  user1      20822 non-null  object 
dtypes: float16(5), float32(1), object(6)
memory usage: 1.2+ MB


In [22]:
res.to_excel('matching_nn2.xlsx')

In [127]:
import time 
start_time=time.time()

res = M.init_matching(13, df_work)

print(time.time()-start_time, 'sec')

C:\Users\EkStepanova\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\EkStepanova\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\EkStepanova\AppData\Local\Continuum\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too sh

530.8642802238464 sec
